In [1]:
from data_manager import Vector, TruncatedMatrix
import json
import pickle
from math import log10 as log
import numpy as np
from scipy import sparse
from sklearn.decomposition import TruncatedSVD

Wczytanie alfabetu i obiektów reprezentujących pojedynczy wektor 'Bag-of-words'

In [2]:
with open("data/alphabet.json", "r", encoding="latin-1") as f:
    alphabet = json.load(f)

In [3]:
vectors: list[Vector] = []
with open("data/Bag_of_words.pickle", "rb") as f:
    while True:
        try:
            vectors.append(pickle.load(f))
        except EOFError:
            break

In [12]:
N = len(vectors)
M = len(alphabet)
print(M, N)

97308 182513


Tworzenie i zapisanie do pliku macierzy(normalnej i inverse document frequency)

In [5]:
word_frequency = {}
for word in alphabet:
    word_frequency[word] = 0

for vec in vectors:
    for word in vec.vector:
        word_frequency[word] += 1

idf = {}
for word in word_frequency:
    idf[word] = log(N / word_frequency[word])

In [31]:
def create_sparse_matrix(filename="sparse_matrix.npz", use_idf=False):
    no_of_entries = 0
    for v in vectors:
        no_of_entries += len(v.vector)

    data = np.array([0 for _ in range(no_of_entries)], dtype=float)
    row = np.array([0 for _ in range(no_of_entries)], dtype=int)
    col = np.copy(row)

    ind = 0
    for i, v in enumerate(vectors):
        start = ind
        for w in v.vector:
            data[ind] = v.vector[w] * (idf[w] if use_idf else 1)
            row[ind] = alphabet[w]
            col[ind] = i
            ind += 1
        data[start:ind] /= np.linalg.norm(data[start:ind])

    matrix_to_save = sparse.csr_matrix((data, (row, col)), shape=(M, N), dtype=float)
    sparse.save_npz(f"data/matrices/{filename}", matrix_to_save)

In [ ]:
create_sparse_matrix()

In [8]:
create_sparse_matrix("sparse_matrix_idf.npz", use_idf=True)

Wczytanie macierzy z pliku

In [4]:
matrix = sparse.load_npz("data/matrices/sparse_matrix.npz")
matrix_idf = sparse.load_npz("data/matrices/sparse_matrix_idf.npz")

Zapisanie obiektów reprezentujących rozkład SVD macierzy idf. Do rozkładu używałem tylko macierzy idf, gdyż miała dużo lepsze wyniki.
Przy svd użyłem pięciu różnych wartości k( k to liczba wartości osobliwych): 20, 50, 100, 200, 400. Na lepsze przybliżenie nie pozwoliłby już RAM

In [6]:
def save_truncated_svd(truncation_level):
    svd = TruncatedSVD(truncation_level).fit(matrix_idf)
    u_s = svd.transform(matrix_idf)
    sigma = svd.singular_values_
    v = svd.components_
    vector_lengths = np.linalg.norm(np.diag(sigma) @ v, axis=0)
    svd_truncation_level = TruncatedMatrix(truncation_level, u_s, sigma, v, vector_lengths)
    with open(f"data/matrices/svd_matrix_{truncation_level}.pickle", "wb") as file:
        pickle.dump(svd_truncation_level, file)

In [17]:
def load_truncated_svd(truncation_level):
    with open(f"data/matrices/svd_matrix_{truncation_level}.pickle", "rb") as file:
        svd = pickle.load(file)
    return svd

In [26]:
%%time
save_truncated_svd(20)

CPU times: total: 26.3 s
Wall time: 32.6 s


In [58]:
%%time
save_truncated_svd(50)

CPU times: total: 49.3 s
Wall time: 55.8 s


In [28]:
%%time
save_truncated_svd(100)

CPU times: total: 1min 30s
Wall time: 1min 38s


In [29]:
%%time
save_truncated_svd(200)

CPU times: total: 2min 15s
Wall time: 2min 13s


In [30]:
%%time
save_truncated_svd(400)

CPU times: total: 4min 4s
Wall time: 3min 33s


### Funkcje obsługujące zapytanie

In [5]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.probability import FreqDist
from string import punctuation
from math import sqrt

stop_words = set([word.lower() for word in stopwords.words('english')])
snowball_stemmer = SnowballStemmer('english')

def normalize_vector(vector):
    length = 0
    for w in vector:
        length += vector[w]**2
    length = sqrt(length)
    for w in vector:
        vector[w] /= length
    return vector

def process_text(text: str) -> dict[str, int]:
    text = "".join(list(map(lambda c: " " if c in punctuation else c, text)))
    words = [w.lower() for sentence in sent_tokenize(text) for w in word_tokenize(sentence)]
    words = [w for w in words if w not in stop_words]
    words = [snowball_stemmer.stem(w) for w in words]
    words = [w for w in words if w in alphabet]
    return normalize_vector(dict(FreqDist(words)))

In [6]:
def convert_to_sparse(query):
    data = np.array([0 for _ in range(len(query))], dtype=float)
    row = np.array([0 for _ in range(len(query))], dtype=int)
    col = np.copy(row)
    ind = 0
    for w in query:
        data[ind] = query[w]
        col[ind] = alphabet[w]
        ind += 1

    return sparse.csr_matrix((data, (row, col)), shape=(1, M), dtype=float)

def k_best_vectors(vector: sparse.csr_matrix):
    best_values = np.sort(vector.todense(), axis=1)[:, ::-1]
    best_indices = np.argsort(vector.todense(), axis=1)[:, ::-1]
    return [(best_indices[0, i], best_values[0, i]) for i in range(k)]

def find_closest_vectors(query):
    sparse_query = convert_to_sparse(query)
    return k_best_vectors(np.abs(sparse_query @ matrix))

def find_closest_vectors_idf(query):
    sparse_query = convert_to_sparse(query)
    return k_best_vectors(np.abs(sparse_query @ matrix_idf))

In [7]:
def convert_to_dense(query):
    data = np.array([0 for _ in range(M)], dtype=float)
    for w in query:
        data[alphabet[w]] = query[w]
    return data

def find_closest_vectors_svd(query, svd: TruncatedMatrix):
    dense_query = convert_to_dense(query)
    best = np.abs(((dense_query @ svd.U_S) @ svd.V) / svd.vector_lengths)
    return list(zip(np.argsort(best)[::-1][:k], np.sort(best)[::-1][:k]))

In [8]:
def print_answers(query_text, function, *args):
    q = process_text(query_text)
    res = function(q, *args)
    url = "https://stackoverflow.com/questions/"
    for i, (ind, angle) in enumerate(res):
        print(f"Result number: {i + 1}")
        print(f"Absolut value of angle between query and result: {angle}")
        print(f"Question: {vectors[ind].question_title}")
        print(f"Question site url: {url}{vectors[ind].question_id}")
        print("--------------------------------------")

## Testowanie różnych wyszukiwarek

W tym przypadku k oznacza ile pierwszych wyników chce wyświetlić

In [9]:
k = 20

Porównanie wyszukiwarek bez svd( idf vs początkowa macierz bag-of-words)

In [38]:
question = "What are generators"

In [39]:
print_answers(question, find_closest_vectors) # bez idf

Result number: 1
Absolut value of angle between query and result: 0.7598468303169561
Question: How to check if an object is a generator object in python?
Question site url: https://stackoverflow.com/questions/6416538
--------------------------------------
Result number: 2
Absolut value of angle between query and result: 0.6980756725957802
Question: Why do Python generator functions not have a syntactically different notation from 'regular' functions?
Question site url: https://stackoverflow.com/questions/20182404
--------------------------------------
Result number: 3
Absolut value of angle between query and result: 0.669251539707131
Question: Generators and for loops in Python
Question site url: https://stackoverflow.com/questions/29570348
--------------------------------------
Result number: 4
Absolut value of angle between query and result: 0.6666666666666666
Question: Redundant use of generators? (Python)
Question site url: https://stackoverflow.com/questions/21025656
-------------

In [40]:
print_answers(question, find_closest_vectors_idf) # idf

Result number: 1
Absolut value of angle between query and result: 0.7091160206580412
Question: How to check if an object is a generator object in python?
Question site url: https://stackoverflow.com/questions/6416538
--------------------------------------
Result number: 2
Absolut value of angle between query and result: 0.6784665317958798
Question: Yield vs generator expression - different type returned
Question site url: https://stackoverflow.com/questions/16780434
--------------------------------------
Result number: 3
Absolut value of angle between query and result: 0.6773114786418795
Question: Generators and for loops in Python
Question site url: https://stackoverflow.com/questions/29570348
--------------------------------------
Result number: 4
Absolut value of angle between query and result: 0.6292592689864435
Question: Can iterator be restored and can its value/status be assigned?
Question site url: https://stackoverflow.com/questions/3211478
------------------------------------

Nie było to zbyt trudne pytanie i odnosiło się do często pojawiającego się tematu, więc obie wyszukiwarki znalazły odpowiednie tematy

In [43]:
question = "is python bytecode much faster than actual python"

In [44]:
print_answers(question, find_closest_vectors) # bez idf

Result number: 1
Absolut value of angle between query and result: 0.5954276895518561
Question: matplotlib plt.show() crashes in mac
Question site url: https://stackoverflow.com/questions/21616774
--------------------------------------
Result number: 2
Absolut value of angle between query and result: 0.5901211295340861
Question: Python/Pyside chrashes without error message
Question site url: https://stackoverflow.com/questions/21048449
--------------------------------------
Result number: 3
Absolut value of angle between query and result: 0.5838318806240852
Question: Python 3.4 crashes when opening an image with PIL in tkinter
Question site url: https://stackoverflow.com/questions/30436924
--------------------------------------
Result number: 4
Absolut value of angle between query and result: 0.5768889424925572
Question: python-crypto for OS X?
Question site url: https://stackoverflow.com/questions/7251676
--------------------------------------
Result number: 5
Absolut value of angle be

In [45]:
print_answers(question, find_closest_vectors_idf) # idf

Result number: 1
Absolut value of angle between query and result: 0.2973842596679014
Question: Python optimization through bytecode
Question site url: https://stackoverflow.com/questions/20940935
--------------------------------------
Result number: 2
Absolut value of angle between query and result: 0.24720598394111326
Question: Python bytecode and .pyc file format specification
Question site url: https://stackoverflow.com/questions/35229387
--------------------------------------
Result number: 3
Absolut value of angle between query and result: 0.24010740836353553
Question: How to validate Python bytecode?
Question site url: https://stackoverflow.com/questions/23267970
--------------------------------------
Result number: 4
Absolut value of angle between query and result: 0.2352181672133219
Question: Is it easy to fully decompile python compiled(*.pyc) files?
Question site url: https://stackoverflow.com/questions/3464326
--------------------------------------
Result number: 5
Absolut v

Jak widać wyszukiwarka bez idf zwróciła coś kompletnie niezwiązanego z tematem, a wartości kątów wynosiły ponad 0.5. W kilku innych wyszukiwaniach można było zauważyć podobne wyniki, gdy wyszukiwarka bez idf znajdywała dokumenty związane jedynie z najpopularniejszym słowem z danego zapytania. Dlatego przy svd używam jedynie wersji z idf.

## Porównanie idf i svd dla różnych poziomów przybliżenia

In [61]:
question = "best python library for data manipulation"

In [62]:
print_answers(question, find_closest_vectors_idf) # idf

Result number: 1
Absolut value of angle between query and result: 0.269410662111676
Question: What am I doing wrong? Python object instantiation keeping data from previous instantiation?
Question site url: https://stackoverflow.com/questions/3161827
--------------------------------------
Result number: 2
Absolut value of angle between query and result: 0.26185907178101075
Question: Merging lines after condition
Question site url: https://stackoverflow.com/questions/23803221
--------------------------------------
Result number: 3
Absolut value of angle between query and result: 0.24871670250028527
Question: Python and Postgresql
Question site url: https://stackoverflow.com/questions/2905097
--------------------------------------
Result number: 4
Absolut value of angle between query and result: 0.24602160276939075
Question: Static library (.lib) to Python project
Question site url: https://stackoverflow.com/questions/3668373
--------------------------------------
Result number: 5
Absolut

In [63]:
truncated_idf = load_truncated_svd(20)

In [64]:
print_answers(question, find_closest_vectors_svd, truncated_idf)

Result number: 1
Absolut value of angle between query and result: 0.11265990558578547
Question: TypeError: a bytes-like object is required, not 'Binary'
Question site url: https://stackoverflow.com/questions/37511560
--------------------------------------
Result number: 2
Absolut value of angle between query and result: 0.1123313855229737
Question: PSD importation library that retains shapes as vectors?
Question site url: https://stackoverflow.com/questions/26235480
--------------------------------------
Result number: 3
Absolut value of angle between query and result: 0.11016458395384122
Question: How to improve a XML import into mongodb?
Question site url: https://stackoverflow.com/questions/21817941
--------------------------------------
Result number: 4
Absolut value of angle between query and result: 0.10997001557042911
Question: Python framework for implementing Open Data Protocol (OData) consumers & producers
Question site url: https://stackoverflow.com/questions/7798432
-------

Przybliżenie rzędu 20 okazało się zdecydowanie za małe, zachowywało zbyt mało informacji o początkowej macierzy i zwracało bardzo losowe wyniki. W większości przypadków było gorsze niż używanie zwykłej macierzy

In [65]:
truncated_idf = load_truncated_svd(50)
print_answers(question, find_closest_vectors_svd, truncated_idf)

Result number: 1
Absolut value of angle between query and result: 0.1311898341503871
Question: How much RAM does data take up?
Question site url: https://stackoverflow.com/questions/20915200
--------------------------------------
Result number: 2
Absolut value of angle between query and result: 0.13075787316126958
Question: Performance comparison of Thrift, Protocol Buffers, JSON, EJB, other?
Question site url: https://stackoverflow.com/questions/296650
--------------------------------------
Result number: 3
Absolut value of angle between query and result: 0.13001187749941215
Question: What is a good storage candidate for soft-realtime data acquisition under Linux?
Question site url: https://stackoverflow.com/questions/13084686
--------------------------------------
Result number: 4
Absolut value of angle between query and result: 0.12882446132758835
Question: How can I use the Python io module to build a memory-resident data structure?
Question site url: https://stackoverflow.com/ques

In [66]:
truncated_idf = load_truncated_svd(100)
print_answers(question, find_closest_vectors_svd, truncated_idf)

Result number: 1
Absolut value of angle between query and result: 0.21204183809173244
Question: Does Python have a rope data structure?
Question site url: https://stackoverflow.com/questions/17005551
--------------------------------------
Result number: 2
Absolut value of angle between query and result: 0.2070540966568405
Question: ETL using Python
Question site url: https://stackoverflow.com/questions/3762199
--------------------------------------
Result number: 3
Absolut value of angle between query and result: 0.19903037108342217
Question: List of big-O analysis for Python datastructures
Question site url: https://stackoverflow.com/questions/4755221
--------------------------------------
Result number: 4
Absolut value of angle between query and result: 0.1949248024761922
Question: Efficient transfer protocol for mobile equipment
Question site url: https://stackoverflow.com/questions/5523889
--------------------------------------
Result number: 5
Absolut value of angle between query 

Wyniki dla przybliżeń rzędu 50 i 100 zdawały się już bardziej odnosić do tematu

In [67]:
truncated_idf = load_truncated_svd(200)
print_answers(question, find_closest_vectors_svd, truncated_idf)

Result number: 1
Absolut value of angle between query and result: 0.33632012569777553
Question: Does Python have a rope data structure?
Question site url: https://stackoverflow.com/questions/17005551
--------------------------------------
Result number: 2
Absolut value of angle between query and result: 0.33164493449974186
Question: Editing a pandas script to ignore but not remove data then match & updating + comparing to prevent wasteful saves + slicing data to match with?
Question site url: https://stackoverflow.com/questions/19938598
--------------------------------------
Result number: 3
Absolut value of angle between query and result: 0.31450157146991625
Question: My function doesn't return a value, can't figure out why
Question site url: https://stackoverflow.com/questions/39156882
--------------------------------------
Result number: 4
Absolut value of angle between query and result: 0.304196318327199
Question: Memory error while slicing an array
Question site url: https://stack

In [68]:
truncated_idf = load_truncated_svd(400)
print_answers(question, find_closest_vectors_svd, truncated_idf)

Result number: 1
Absolut value of angle between query and result: 0.37596041565212107
Question: Editing a pandas script to ignore but not remove data then match & updating + comparing to prevent wasteful saves + slicing data to match with?
Question site url: https://stackoverflow.com/questions/19938598
--------------------------------------
Result number: 2
Absolut value of angle between query and result: 0.3645996003363533
Question: Does Python have a rope data structure?
Question site url: https://stackoverflow.com/questions/17005551
--------------------------------------
Result number: 3
Absolut value of angle between query and result: 0.3551919851016043
Question: List and += operation
Question site url: https://stackoverflow.com/questions/33950163
--------------------------------------
Result number: 4
Absolut value of angle between query and result: 0.3391932552025241
Question: Memory error while slicing an array
Question site url: https://stackoverflow.com/questions/13915381
----

Dla trudniejszego zapytania idf nie zwróciło żadnych sensownych informacji, dopiero przy przybliżeniu rzędu 400 pojawiły się wzmianki o jakieś faktycznej bibliotece
(pandas)

In [69]:
question = "How to write a search engine in python"

In [70]:
print_answers(question, find_closest_vectors_idf) # idf

Result number: 1
Absolut value of angle between query and result: 0.4343968817518417
Question: SQLalchemy with nested class
Question site url: https://stackoverflow.com/questions/4172427
--------------------------------------
Result number: 2
Absolut value of angle between query and result: 0.4041230793069078
Question: Best full-text search for google-app-engine
Question site url: https://stackoverflow.com/questions/2679601
--------------------------------------
Result number: 3
Absolut value of angle between query and result: 0.4013177559867625
Question: Full text search with google app engine using ndb models
Question site url: https://stackoverflow.com/questions/9239575
--------------------------------------
Result number: 4
Absolut value of angle between query and result: 0.3580153653917563
Question: Using Search API Python - Google App Engine Big Table
Question site url: https://stackoverflow.com/questions/13305302
--------------------------------------
Result number: 5
Absolut va

In [71]:
print_answers(question, find_closest_vectors_svd, truncated_idf) # k = 400

Result number: 1
Absolut value of angle between query and result: 0.41110074125940893
Question: Advanced python string search
Question site url: https://stackoverflow.com/questions/7143418
--------------------------------------
Result number: 2
Absolut value of angle between query and result: 0.404309907412277
Question: Google App Engine Datastore Faceted Search
Question site url: https://stackoverflow.com/questions/2501431
--------------------------------------
Result number: 3
Absolut value of angle between query and result: 0.399569762005354
Question: Prospective Search Location-Based Searches
Question site url: https://stackoverflow.com/questions/14266314
--------------------------------------
Result number: 4
Absolut value of angle between query and result: 0.38637871782203476
Question: Full text search with google app engine using ndb models
Question site url: https://stackoverflow.com/questions/9239575
--------------------------------------
Result number: 5
Absolut value of angl

W tym przypadku oba modele zwróciły dość dobre wyniki. Poza informacjami o różnych wyszukiwarkach w odpowiedziach znalazły się różne poradniki.

In [88]:
question = "Switch case in python"

In [89]:
print_answers(question, find_closest_vectors_idf) # idf

Result number: 1
Absolut value of angle between query and result: 0.5084885998423402
Question: Resolving shift/reduce conflict for default label in switch block
Question site url: https://stackoverflow.com/questions/39797216
--------------------------------------
Result number: 2
Absolut value of angle between query and result: 0.4797240600090062
Question: Is there any value to a Switch / Case implementation in Python?
Question site url: https://stackoverflow.com/questions/5440990
--------------------------------------
Result number: 3
Absolut value of angle between query and result: 0.457133947818973
Question: Is this a "pythonic" method of executing functions as a python switch statement for tuple values?
Question site url: https://stackoverflow.com/questions/7857837
--------------------------------------
Result number: 4
Absolut value of angle between query and result: 0.419417932636208
Question: Replacements for switch statement in Python?
Question site url: https://stackoverflow.c

In [90]:
print_answers(question, find_closest_vectors_svd, truncated_idf)

Result number: 1
Absolut value of angle between query and result: 0.052003502749733706
Question: Should I optimise my python code like C++? Does it matter?
Question site url: https://stackoverflow.com/questions/1353715
--------------------------------------
Result number: 2
Absolut value of angle between query and result: 0.05137634195540674
Question: Necessity of an 'empty' else statement in Python
Question site url: https://stackoverflow.com/questions/31575251
--------------------------------------
Result number: 3
Absolut value of angle between query and result: 0.05009837152788884
Question: another switch case with functions
Question site url: https://stackoverflow.com/questions/4681556
--------------------------------------
Result number: 4
Absolut value of angle between query and result: 0.04974677540559106
Question: "else" considered harmful in Python?
Question site url: https://stackoverflow.com/questions/865741
--------------------------------------
Result number: 5
Absolut va

Dla tego prostego zapytania jednak svd zwróciło kilka dziwnych niezwiązanych z tematem wyników. A także maksymalna wartość kąta była bardzo mała

### Wnioski

Najważniejszą częścią było użycie inverse document frequency. Bez niej model zwracał uwagę jedynie na najważniejsze słowa w każdym tekście.
SVD dawało czasami odpowiedzi, które były kontekstowo związane z danym zapytaniem, lecz zwracało też czasami wyniki wgl z tym niezwiązane.
Zwykła macierz była bardziej "stabilna", lecz dla trudniejszych zapytań nie było szans na znalezienie sensownych odpowiedzi.
Ogółem SVD nie dawało bardzo dużej poprawy. Możliwe, że wybrałem nieodpowiedni dataset. Dokumentów było 180 tysięcy, więc można było spodziewać się odpowiedzi na większość pytań odnośnie pythona, lecz po tokenizacji i przetworzeniu dokument średnio zawierał około 112 słów z alfabetu i być może było to zbyt mało i ciężko było znaleźć korelację dla bardzo rzadkich wektorów powstałych z danego zapytania

